## 准备 GEE 环境

In [0]:
#%% 设置本机代理，根据自己电脑 FQ 的代理来填写，若使用 Colab 则不需要
import os
os.environ['http_proxy'] = "http://127.0.0.1:1087"
os.environ['https_proxy'] = "https://127.0.0.1:1087"

In [0]:
#%% 初始化 Google Earth Engine
import ee
# ee.Authenticate() # 在 Colab 上初次使用的话需配置此项
ee.Initialize()

## 配置需要请求的数据集

In [0]:
#%% 构造 TROPOMI 数据集请求
#
#
class getTROPOMI(object):
    def __init__(self, startDate, endDate):
        # startDate 是闭区间 '2020-02-01'
        # endDate 是开区间 '2020-02-03'
        # 取到的是 2 月 1 日和 2 日的数据
        self.sD = startDate
        self.eD = endDate

    def NO2_offl(self):
        collection = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2')\
            .select('tropospheric_NO2_column_number_density')\
            .filterDate(self.sD, self.eD)
        return collection

    def SO2_offl(self):
        collection = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_SO2')\
            .select('SO2_column_number_density')\
            .filterDate(self.sD, self.eD)
        return collection

    def CH4_offl(self):
        collection = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CH4')\
            .select('CH4_column_volume_mixing_ratio_dry_air')\
            .filterDate(self.sD, self.eD)
        return collection

In [0]:
#%% 构造 MODIS 数据集请求
#
#
class getMODIS(object):
    def __init__(self, startDate, endDate, type):
        # startDate 是闭区间 '2020-02-01'
        # endDate 是开区间 '2020-02-03'
        # 取到的是 2 月 1 日和 2 日的数据
        self.sD = startDate
        self.eD = endDate
        self.type = type

    def qa_MAIAC(self): # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<这里还没有修改
        collection = ee.ImageCollection('MODIS/006/MCD19A2_GRANULES')\
            .select('AOD_QA')\
            .filterDate(self.sD, self.eD)
        return collection

    def MAIAC(self): # 这里用的是 aod550，可以更改
        collection = ee.ImageCollection('MODIS/006/MCD19A2_GRANULES')\
            .select('Optical_Depth_055')\
            .filterDate(self.sD, self.eD)
        return collection

    def qa_vi(self,image):
        qa = image.select('SummaryQA')
        mask = qa.eq(0) # 这里只做了最严格的筛选
        bands = image.select(['NDVI','EVI'])
        return bands.updateMask(mask)

    def VI(self):
        if self.type == 'aqua':
            VI_name = 'MODIS/006/MYD13Q1'
        if self.type == 'terra':
            VI_name = 'MODIS/006/MOD13Q1'

        collection = ee.ImageCollection(VI_name)\
                       .filterDate(self.sD, self.eD)
        return collection.map(self.qa_vi)
    
    def qa_lst(self,image):
        qa_day = image.select('QC_Day')
        qa_night = image.select('QC_Night')
        mask_day = qa_day.eq(0)
        mask_night = qa_night.eq(0)
        band_day = image.select('LST_Day_1km').updateMask(mask_day)
        band_night = image.select('LST_Night_1km').updateMask(mask_night)
        return ee.Image.cat([band_day, band_night])

    def LST(self):
        if self.type == 'aqua':
            LST_name = 'MODIS/006/MYD11A1' # MYD11A2
        if self.type == 'terra':
            LST_name = 'MODIS/006/MOD11A1' # MOD11A2
        collection = ee.ImageCollection(LST_name)\
                    .filterDate(self.sD, self.eD)
        return collection.map(self.qa_lst)

In [0]:
#%% 构造影像请求
def get_ImageURL(collection,name,region,scale=1113.1949079327358,method='mean'):
    # collection 数据集
    # name 压缩包里面文件的名字
    # region 需要下载的区域
    # scale 分辨率（以米为单位）默认换算下来为0.01°
    # method 默认为均值 mean，也可改为获取数据集的第首个影像 first
    image = collection.mean()
    if method == 'first':
        image = collection.first()
    if method == 'stdev':
        image = collection.reduce(ee.Reducer.stdDev())
    url = image.getDownloadURL({'name':name,'region':region,'scale':scale})
    return url

## 配置返回的数据空间范围和时间范围

In [0]:
#  框选地理范围
roi = ee.Geometry.Polygon(
        [[[106.1, 37],
          [107, 37],
          [107, 38.8],
          [106.1, 38.8]]])
roiGeo = ee.Geometry(roi).toGeoJSONString()

In [0]:
# %%
import pandas as pd
from datetime import datetime
#  选择日期范围
# dateRange = pd.date_range('2018-12-31','2019-12-31',freq='Y')+ pd.DateOffset(days=1) #逐年
# dateRange = pd.date_range('2018-11-30','2019-12-01',freq='M')+ pd.DateOffset(days=1) #逐月
dateRange = pd.date_range('2018-11-30','2020-04-01',freq='3M')+ pd.DateOffset(days=1)# 逐季
# dateRange = pd.date_range('2018-07-01','2020-03-01') # 逐日
dateList = [datetime.strftime(d,"%Y-%m-%d") for d in dateR

# dateList = ['2019-03-01','2020-03-01','2020-05-01'] #自定义时间范围ange]

## 请求影像地址并下载
根据要下载的类型获取数据集->影像下载链接->下载解压

In [0]:
for i in range(len(dateList)-1):
    sD = dateList[i] # 开始时间
    eD = dateList[i+1] # 结束时间
    # #%% 开始和结束时间格式如下
    # sD = '2019-01-01'
    # eD = '2019-01-02'
    print(sD,eD)

    #  构造实例
    g = getMODIS(sD,eD,'aqua') # <<<<<<<<<<<<<<<<<<<<<<<<<<<注意选择传感器对应数据集
    get = g.LST() # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<注意选择需要的参数
    url = get_ImageURL(get,name=sD.split('-')[0],region=roiGeo,scale=250,method='mean')#<<<<<<<<注意选择合适的文件名
    !wget {url} -O ./tmp.zip && unzip tmp.zip -d ./

TROPOMI

In [0]:
for i in range(len(dateList)-1):
    sD = dateList[i] # 开始时间
    eD = dateList[i+1] # 结束时间
    # #%% 开始和结束时间格式如下
    # sD = '2019-01-01'
    # eD = '2019-01-02'
    print(sD,eD)

    #  构造实例
    g = getTROPOMI(sD,eD) # <<<<<<<<<<<<<<<<<<<<<<<<<<<注意选择传感器对应数据集
    get = g.NO2_offl() # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<注意选择需要的参数
    url = get_ImageURL(get,name=sD.replace('-',''),region=roiGeo,method='mean')#<<<<<<<<注意选择合适的文件名
    !wget {url} -O ./tmp.zip && unzip tmp.zip -d ./

## 处理生成的 tif 文件

重新改为使用 rioxarray 来转换数据

In [ ]:
!pip install rioxarray

In [ ]:
import rioxarray
xds = rioxarray.open_rasterio(
    "/content/20190301.CH4_column_volume_mixing_ratio_dry_air.tif",
    masked=True,
    # chunks=True,
)

xds = xds.where(xds>0,drop=True) # CH4

xds.plot()
xds.to_netcdf('20190301.nc')

需要安装 rasterio 库

In [0]:
!pip install rasterio

### 根据类型选择要处理的文件

In [0]:
import rasterio
import xarray as xr
import numpy as np
import pandas as pd
from pathlib import Path

feature = 'NO2' # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<注意选择

# 获取 feature 的所有 tif 文件
filePath = Path('./')
if feature == 'NDVI':
    fileList = list(filePath.glob('*NDVI.tif'))
if feature == 'EVI':
    fileList = list(filePath.glob('*EVI.tif'))
if feature == 'LST_Day':
    fileList = list(filePath.glob('*LST_Day_1km.tif'))
if feature == 'LST_Night':
    fileList = list(filePath.glob('*LST_Night_1km.tif'))
if feature == 'NO2':
    fileList = list(filePath.glob('*tropospheric_NO2_column_number_density.tif'))
if feature == 'SO2':
    fileList = list(filePath.glob('*SO2_column_number_density.tif'))
fileList.sort()
fileList

### 将 tif 文件转为 xarray 格式

In [0]:
# 建立空列表，后续用来存储值和时间
all_val = []
all_time = []

for f in fileList[:]:
    print(f)
    # 先用 rasterio 读取 tif 文件的相关信息
    dataset = rasterio.open(f)
    # 1. 坐标范围信息
    db = dataset.bounds
    # 计算出格网的边缘点数
    line_Lat = np.linspace(db.bottom, db.top, num=dataset.height + 1) # 加的 1 是边缘点， dataset.height 是格网数
    line_Lon  = np.linspace(db.left, db.right, num=dataset.width + 1)
    # 计算出格网的中心点坐标
    center_Lat = (line_Lat[1:]+line_Lat[:-1])/2
    center_Lon = (line_Lon[1:]+line_Lon[:-1])/2
    # 2. 数值信息
    if feature == 'NO2':
        tmp_val = dataset.read(1) * 6.02214E+4 # 将mol/m2转为10^15 molec./cm2
        tmp_val = np.where(tmp_val<=0,np.nan,tmp_val)
    if feature == 'SO2':
        tmp_val = dataset.read(1) * 6.02214E+4 # 将mol/m2转为10^15 molec./cm2
        tmp_val = np.where(tmp_val<=0,np.nan,tmp_val)

    if feature == 'NDVI':
         tmp_val = dataset.read(1) * 0.0001
         tmp_val = np.where(tmp_val==0,np.nan,tmp_val)

    if feature == 'EVI':
         tmp_val = dataset.read(1) * 0.0001
         tmp_val = np.where(tmp_val==0,np.nan,tmp_val)
         
    if feature == 'LST_Day':
         tmp_val = dataset.read(1) * 0.02
         tmp_val = np.where(tmp_val==0,np.nan,tmp_val)

    if feature == 'LST_Night':
         tmp_val = dataset.read(1) * 0.02
         tmp_val = np.where(tmp_val==0,np.nan,tmp_val)

         

    all_val.append(tmp_val)

    tmp_time = f.stem.split('.')[0]
    all_time.append(tmp_time)

timeArray = pd.to_datetime(all_time, format='%Y-%m-%d') 
#%%
ds = xr.Dataset( {
    f'{feature}': (( 'time','latitude', 'longitude'), all_val),
                 },
    coords= { 
            'time':timeArray,
            'latitude': center_Lat[::-1], 
            'longitude': center_Lon
            }
               )

### 检查数据并保存，若有需要可以重采样至需要的范围

In [0]:
ds[feature].isel(time=0).plot()

In [0]:
# 插值到统一的范围
ds_i = ds.interp(longitude=np.array(range(450))*0.002 + 106.1,\
                 latitude=np.array(range(850))*0.002 + 37.1)
ds_i.to_netcdf(filePath/(feature+'.nc'))

In [0]:
!rm *.tif